In [11]:
import numpy as np
from keras import layers
from load_data import load
print('done importing')

done importing


In [13]:
[X,Y] = load("blast_tab_1hit.out")

490670 sequences were uploaded
431807 sequences had the same input and output
58863 had errors

Maximum sequence length in is 351
Maximum sequence length out is 351

Converting to one-hot...
Done


In [14]:
print(X[200],Y[200])

[[1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
